# Ethereum: Blockchain 2.0

> Computational Analysis of Social Complexity
>
> Fall 2021, Spencer Lyon

**Prerequisites**

- Intro to Blockchain

**Outcomes**

- Recall the key components of the Bitcoin blockchain
- Understand the key features of a finite state machine
- Understand the additional features Ethereum brought to the blockchain
- Code up some sample smart contracts

**References**

- https://ethereum.org/en/whitepaper/

## Review: What is a Blockchain?

- A blockchain is a **decentralized distributed ledger**:
    - Ledger: record of financial transactions
    - Distributed: full copies of ledger stored on multiple (all) nodes
    - Decentralized: no single point of trust, control, or failure for network

### Review: How does the Blockchain Work?

- Users construct and **sign** transactions
- Transactions are **submitted** to nodes (miners)
- Nodes collect a bundle of transactions and order them in a **block** and do:
    1. **validate** all transactions in block (check fund ownership and **verify** signatures)
    2. Find a **proof of work** (PoW -- key to block that produces hash with special properties)
-  First miner to find PoW gets BTC as reward, other nodes verify and all move to next block
- Hash of block N-1 included in header of block N (any change to historical blocks changes all subsequent hashes and invalidates blocks) ==> block chain/

### Review: Properties of Blockchain

- Trustless: to interact with another individual, I don't have to trust them or any third party
- Permissionless: permission can neither be granted nor revoked -- everyone on equal footing in terms of access
- Censorship resistant: cannot be manipulated or controlled by any single party or entity

### Review: Issues with Blockchain

- PoW requires extreme amount of computation => uses energy => damages environment
- Limited throughput
- "Just" a ledger -- can only record transfers of BTC (with *minor* caveat)

## Key Abstraction: State Machine

- In computer science, a **state machine** is a system with the following properties
    - Can be in exactly one of a (usually finite) number of "states" at any time
    - Responds to *inputs* to *transition* from one state to another state
- Let 
    - $s$: state of the machine at some moment
    - $tx$: an arbitrary set of inputs
    - $s'$ the "next" state after processing $tx$
    - $f(s, tx) = s'$: the transition function that produces the next state ($s'$) given current state ($s$) and an input set ($tx$)

### Bitcoin as State Machine

- In a blockchain the state is a record of all unspet transaction outputs (**UTXO**)
     - UTXO is a more accurate technical term for "account balance". It is ok to think of this as the number of coins held by an account, or their balance
- Each transaction modifies the state of the blockchain
    - Example: Leslie gives Ron 2.3 BTC results in 
    ```
    0xLeslie000 -= 2.3 BTC
    0xRon000000 += 2.3 BTC
    ```
- The Bitcoin blockchain (mostly) limits the transition function $f$ to modify account balances

### Ethereum's Innovation

- The Ethereum project took the concept of treating the blockchain as a state machine one step further...
    - Each node runs a turing complete virtual machine (called Ethereum virtual machine or EVM)
    - Transactions can now encode arbitrary logic, expressed in bytecode of EVM
    - Code executed during transaction validation stage
    - Makes validation slightly more expensive, but cost is negligible compared to proof of work (which Ethereum still uses)

## Ethereum: History

- [Whitepaper](https://ethereum.org/en/whitepaper/) published in 2013
- [Officially launched](https://ethereum.org/en/history/) in 2015
- Founded by Vitalik Buterin (considered main founder), Anthony Di Iorio, Charles Hoskinson, Mihai Alisie & Amir Chetrit
- Many more people have worked on it
- Has been 2nd largest (only bitcoin is biggest) in terms of Market cap for many years
- It's coin (Ether, short ETH) is largest cryptocurrency based on settlement value (bigger than BTC) -- on track to settle over [8 trillion USD in 2021](https://twitter.com/RyanWatkins_/status/1415314367860682756)
- Home blockchain to many, many projects
    - If you add up market cap for ETH + all tokens built on Ethereum, about same size as BTC

### Smart Contracts

- The ethereum blockchain has two types of accounts
    1. Externally owned accounts: these are simlar to bitcoin accounts. Held/controlled by a public/private key
    2. Contract Accounts (smart contracts): can hold ETH and interact with other accounts, but controlled by contract code
- Contracts can receive messages and execute pre-defined instructions in response

### What is a Smart Contract?

- I like to think of smart contracts like classes in a programming language
- Like a class, a smart contract
    - Has a name
    - Can be "instantiated" (deployed)
    - Has properties or data
    - Has actions/functions that act on properties and/or externally supplied data at call time
- Unlike a class, a smart contract
    - Is immutable -- once it is deployed it can't be changed
    - Is `payable` (can receive tokens/coins)
    - Is stored on the blockchain -- the code defining the contract's functionality is stored as the data field on the contract account

### Example: Safe Online Purchase

- Suppose you would like to purchase a one-of-a-kind piece of art  directly from the artist
- Because of COVID-19, this artist does all transactions online
- To make the purchase you must trust
    1. The payment provider used by the artist
    2. The artist to actually deliver the artwork
- The typicaly flow of events is:
    1. You identify good and send money to payment provider
    2. Payment provider gives confirmation of receipt to artist
    3. Artist sends you the goods
    4. (not necessarily in this order), artist gets money from payment provider

### ... continued

- Because of the high price, this is a pretty risky transaction
- You and the artist agree upon an altered system for how this works
- When you decide to purchase an item both you and the artist must send 2x the sale price to the payment provider
- Once the payment provider gets all the funds and seller has shipped product, the following happens
    - Seller confirms reciept of goods
    - Payment provider sends 1/2 buyer's money to buyer
    - Payment provider sends 1/2 buyer's money + all of seller's money to seller
- The idea is that both parties have an incentive to complete the transaction, or they forfeit their deposits
- This is an improvement, but you now have a lot of trust in payment provider...

### ... continued

- You and the artist happen to be programmers
- To help the payment provider, you give them the following Julia code to help them fulfill their duties:

In [5]:
@enum PurchaseState begin
    Created
    Locked
    Released
    Inactive
end

mutable struct Purchase
    value::Number
    buyer_deposit::Int
    seller_deposit::Int
    good::Any
    state::PurchaseState
    
    # start a new purchase -- initiated
    # by buyer sending an initial deposit for the item
    function Purchase(buyer_deposit::Int)
        new(buyer_deposit/2, buyer_deposit, 0, nothing, Created)
    end
end

"""
Seller confirms purchase and sends their deposit.

To be called only by seller!
"""
function confirm_purchase(p::Purchase, seller_deposit::Int, good::Any)
    # TODO: how to guarantee seller is caller??
    
    # make sure deposits match
    @assert seller_deposit == p.buyer_deposit

    # make sure state is created
    @assert p.state == Created
        
    # record deposit
    p.seller_deposit = seller_deposit
    
    # send good
    p.good = good
    
    # note that the state is locked
    p.state = Locked
end

"""
Buyer confirms receipt

To be called only by buyer!
"""
function confirm_receipt(p::Purchase)
    # TODO: how to make sure buyer is caller??
    # assert state is locked
    @assert p.state == Locked
    
    # mark good as having been released
    p.state = Released
    
    # send 1/2 deposit back to buyer
    p.buyer_deposit -= p.value
end

"""
Seller gets payment

To be called only be seller!
"""
function finalize_transaction(p::Purchase)
    # TODO: how to make sure seller is caller
    @assert p.state == Released
    
    # send rest of buyer's deposit and all of seller's deposit
    p.buyer_deposit -= p.value
    p.seller_deposit -= 2*p.value
    
    p.state = Inactive
end

finalize_transaction

### ... continued

- Let's try out our code
- Suppose the purchase price is $2,000
- The item is a very nice string

In [11]:
# start transaction with buyer sending $4000 = 2x price
item = "Happy Holidays!"
p = Purchase(4000)
p

Purchase(2000.0, 4000, 0, nothing, Created)

In [12]:
confirm_purchase(p, 4000, item)
p

Purchase(2000.0, 4000, 4000, "Happy Holidays!", Locked)

In [13]:
confirm_receipt(p)
p

Purchase(2000.0, 2000, 4000, "Happy Holidays!", Released)

In [14]:
finalize_transaction(p)
p

Purchase(2000.0, 0, 0, "Happy Holidays!", Inactive)

### ...comments

- This code will help the payment provider make sure no operations happen out of sequence
    - If seller tries to finalize before buyer confirms receipt, code will throw an error
    - If buyer tries to confirm receipt before seller provides good and deposit, code errors
    - If anyone tries to do their transaction twice, error
- There are still problems with the code
    - Some function should only be called by either seller or buyer, but we haven't enforced that
    - Code doesn't actualy do value transfer, still trust payment provider to do steps as we agreed

### Example: Purchase as Smart Contract

- Let's revisit this example, but using a Smart Contract
- For that we'll open up terminal + text editor and look at the file "Purchase.sol"
- For reference, these are the commands I enter at the console prompt:
```python
buyer, seller = accounts[:2]
price = 2000
p = Purchase.deploy({"from": buyer, "value": price*2})
p.confirmPurchase({"from": seller, "value": price*2})
p.confirmReceived({"from": buyer})
p.finalizePurchase({"from": seller})
```

### Notes

- Each time we call a function, we did an ethereum transaction
- A transaction receipt contains the hash of the transaction and other helpful information

### Transaction Data: Events

- Contract method calls are transactions with:
    - Caller (`msg.sender`)
    - Amount of Ether (`msg.value`)
    - Inputs (function arguments)
    - Outputs (return values)
    - Emitted Events
- The Event system in Ethereum works as follows:
    - Event types are defined within the SmartContract (see `event` keyword in our Purchase.sol contract)
        - They have a name, and some parameters
    - When contract methods are called, events are `emit`ted
    - Emitted events are collected/reported alongside transaction

### Transaction Data: Logs

- Whenever an event is emitted from a smart contract, a `log` is created -- there is a 1:1 relationship between emit and log
- These logs are the main visibility window into the data passing through contracts
- A team building a smart contract system with web-based interface will
    - Write contracts to fulfill goals
    - Craft event types to capture key semantic data
    - Emit events in the smart contract methods
    - Process the logs that are generated as users interact with platform
    - Display the processed log data in front-end

### Example: Uniswap V3 Info

- Uniswap is a smart contract system that allows users to swap one ethereum based token for another one
- We'll learn much more about it in a future lecture
- For now we'll focus only on a specific type of event: [`Swap`](https://github.com/Uniswap/v3-core/blob/234f27b9bc745eee37491802aa37a0202649e344/contracts/interfaces/pool/IUniswapV3PoolEvents.sol#L64-L80)
- The `Swap` event is emitted every time a trade or swap happens on the platform
- The Uniswap team uses it to track things like:
    - Total number of swaps
    - Swaps per user
    - Swaps per coin
    - Price of coins
    - Volume in USD of swaps
    - Etc.
- We can see all this processed log data on their data dashboard: https://info.uniswap.org/#/

### Data Sources

- There are a few ways to get log data from Ethereum transactions:
    1. Access via Ethereum node. This is the ultimate source of truth and will let you access everything. It is also the most technical
    2. [TheGraph](https://thegraph.com/en/): project that allows developers to specify how to process log data and creates a data marketplace for data indexing/access
    3. Other, private APIs: [Bitquery](https://bitquery.io), [etherscan](https://etherscan.io), [coingecko](https://www.coingecko.com/en), [coinmarketcap](https://coinmarketcap.com/api/), many more...

## Additional Examples

- Below we have a couple more examples of what smart contracts could be used for
- This *barely* scratches the surface of what is possible
- The goal is to whet your appetite to learn more

### Example: Escrow

- Suppose you want to purchase a home
- The typical flow of events includes:
    1. Buyer makes an offer
    2. Seller accepts
    3. Buyer deposits some money into an escrow account
    4. Insepections are done, contracts drafted
    5. Seller gives title (ownership) to escrow agent
    6. Buyer gives escrow agent rest of down payment + closing costs + mortgage information
    7. When (and only when) escrow agent has both title from seller and all funds from buyer, makes swap to transfer funds to seller and title to buyer

### Escrow in contracts

- The escrow example is simlar to our purcahse example
- Here the good is a home, represented by the title
- The best scenario is that the title is **tokenized** and represented on the blockchain
- In this case the contract would need to:
    1. Collect escrow funds from buyer
    2. Collect title from seller
    3. Collect closing costs/downpayment/mortgage info from buyer
    4. Swap title for funds
- The benefit of doing this on the blockchain is that there is no need for an Escrow agent (who takes a fee) or trust between parties. 

### Example: Lending and Borrowing

- Suppose you hold some ETH, don't want to sell it, but need USD to pay bills
    - You could take our ETH to a smart contract based lending platform like Aave or compound and deposit as collateral
    - Then you can borrow up to 1/2 the value of your collateral as USDC
    - You can take USDC back to your exchange and get USD
    - Pay your bills using the USD
    - When you get a paycheck you go pack to exchange to convert USD -> USDC
    - Then you go back to Aave, repay your loan, and reclaim your ETH
- Amazing thing is:
    1. You don't need credit or permisison to get loan (your ETH is your credit)
    2. You don't have to apply
    3. you don't need to know your lenders
    4. System protects lenders -- if your collateral falls in value, Aave's smart contracts will sell it to pay back lenders
    5. No fixed duration/terms/due date

## Looking Ahead

- The possibilities with smart contracts on a blockchain are vast and still being explored
- We have two more weeks of class
    1. Study decentralized finance, the ERC-20 token standard, and Dapps like Uniswap/Curve/Aave
    2. Study the ERC-721 token standard, NFTs, and Dapps like OpenSea/TopShots/Rarible/AxieInfinity